# Instalando dependências

In [1]:
!pip install ket-lang
!ket --version
from ket import *
from ket.lib import qft
from ket.plugins import pown
from random import randint, choice
from functools import reduce
from math import log2, gcd
from string import ascii_lowercase, digits, punctuation
from hashlib import sha256
import json
import time
import rsa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Ket 0.5.0.1


## Funções para a assinatura digital utilizando o RSA

In [2]:
alfa = ascii_lowercase + digits + punctuation

def modInverse(A, M):

    for X in range(1, M):
        if (((A % M) * (X % M)) % M == 1):
            return X
    return -1


def generate_key(p: int, q: int):
    # Pre-requisites
    # p and q are prime
    # p != q
  

    # Choose e such that gcd(e, phi_n) == 1.
    phi_n = (p - 1) * (q - 1)

    # Since e is chosen randomly, we repeat the random choice
    # until e is coprime to phi_n.
    e = randint(2, phi_n - 1)
    while gcd(e, phi_n) != 1:
        e = randint(2, phi_n - 1)

    # Choose d such that e * d % phi_n = 1.
    d = modInverse(e, phi_n)

    return (e, d)

def encrypt(plain_text, e, N):
    cipher_text = ""
    for letter in plain_text:
        k = (alfa.find(letter) ** e) % N
        k_= alfa[k]
        cipher_text += k_
    return cipher_text

def decrypt(cipher_text, d, N):
    plain_text = ""
    for letter in cipher_text:
        # k < n
        # Indice da letra tem que ser menor que o produto de p e q (N)
        k = (alfa.find(letter) ** d) % N
        k_ = alfa[k]
        plain_text += k_
    return plain_text

## Funções para a blockchain

In [3]:
blockchain = []
transactions = []

def send_transaction(transaction, d, N):
	plain_text = transaction['remetente'] + transaction['destinatario'] + transaction['valor'] 
	transaction["signature"] = encrypt(plain_text, d, N)
	print("Sending transaction: ", transaction['remetente'], "pays", transaction['valor'], "to", transaction['destinatario'], )
	transactions.append(transaction)

def add_transaction_to_block():
	block = dict()
	if len(blockchain) != 0:
		block["previous_hash"] = blockchain[-1]['hash']
	block["transactions"] = tuple(transactions)
	block["hash"] = sha256(json.dumps(block).encode('utf-8')).hexdigest()
	blockchain.append(block)
	transactions.clear()
	print("Block added")

def verify_transaction(name, pb, N):
	for block in blockchain:
		for trans in block['transactions']:
			if trans['destinatario'] == 'ana':
				aux = trans['remetente'] + trans['destinatario'] + trans['valor'] 
				sig = trans['signature']
				plain_text = decrypt(sig, pb, N)
				if plain_text == aux:
					print(f'{name} verifiou a transacao')

def create_blockchain():
	for i in range(3):
		name = "genesis"
		value = randint(1, 100)
		send_transaction({"remetente": name, "destinatario": name, "valor": str(value)}, 7, 5)
	add_transaction_to_block()


def print_blockchain():
	for block in blockchain:
		print()
		for key, value in block.items():
			if key == "previous_hash":
				color = "\033[91m {}\033[00m"
				print(color.format(key), " : ", value)
			elif key == "transactions":
				color = "\033[92m {}\033[00m"
				print(color.format(key), " : ")
				print_transaction(value)
			else:
				color = "\033[95m {}\033[00m"
				print(color.format(key), " : ", value)

			#print(color.format(key), " : ", value)
			#print(key, ' : ', value)

def print_transaction(t):
	for i in t:
		print("	", i["remetente"], "pays", i["destinatario"], i["valor"])
			

## Funções para o algoritmo de shor

### Subrotina Quântica

In [4]:
def quantum_subroutine(N, x):
    n = N.bit_length()

    def subroutine():
        reg1 = H(quant(n))
        reg2 = pown(x, reg1, N)
        measure(reg2)
        adj(qft, reg1)
        return measure(reg1).value

    lista = [subroutine() for _ in range(n)]
    r = reduce(gcd, lista)
    return 2**n//r

In [5]:
def shor(N: int) -> int:
    if N % 2 == 0:
        return 2
    n = N.bit_length()
    y = int(log2(N))
    for b in range(2, n+1):
        x = y/b
        u1 = int(2**x)
        u2 = u1+1
        if u1**b == N:
            return u1
        elif u2**b == N:
            return u2

    for _ in range(N.bit_length()):
        x = randint(2, N-1)
        gcd_x_N = gcd(x, N)
        if gcd_x_N > 1:
            return gcd_x_N

        r = quantum_subroutine(N, x)

        if r % 2 == 0 and pow(x, r//2) != -1 % N:
            p = gcd(x**(r//2)-1, N)
            if p != 1 and p != N and p*N//p == N:
                factor = p
                break

            q = gcd(x**(r//2)+1, N)
            if q != 1 and q != N and q*N//q == N:
                factor = q
                break

    if factor is not None:
        return factor
    else:
        raise RuntimeError(f"fails to factor {N}")

# Aplicação: Ataque as transações Ethereum

## 1. Instanciando a blockchain

In [ ]:
create_blockchain()
print_blockchain()



## 2. Alice gera seu par de chaves

In [ ]:
alice_p = 5
alice_q = 11
alice_N = alice_p * alice_q
alice_pb, alice_pv = generate_key(alice_p, alice_q)
while alice_pb == alice_pv:
    alice_pb, alice_pv = generate_key(alice_p, alice_q)
print("Chave pública: ", alice_pb)
print("Chave privada: ", alice_pv)

## 3. Alice gera sua transação e manda para o validador (= minerador da bitcoin)

In [ ]:
# Alice gera seu bloco
alice_transaction ={"remetente": "alice", 'destinatario': 'bob', 'valor': '150'}
send_transaction(alice_transaction, alice_pb, alice_N)

## 4. Mais algumas transações

In [ ]:

# Alguns números primos
prime = [3, 5, 7, 11]

# Alguns nomes
names = ["bob", "jeff", "bill"]

for i in range(randint(1, 3)):
    # Bob gera seu par de chaves
    bob_p = choice(prime)
    bob_q = choice(prime)
    while bob_p == bob_q:
        bob_q = choice(prime)
    bob_N = bob_p * bob_q
    bob_pb, bob_pv = generate_key(bob_p, bob_q)

    # Bob gera seu bloco
    bob_block = {"remetente": choice(names), 'destinatario': choice(names), 'valor': str(randint(1, 150))}
    send_transaction(bob_block, bob_pv, bob_N)

## 5. Minerador adiciona a transação de Alice dentro de um bloco


In [ ]:
# Adiciona seu bloco na blockchain
add_transaction_to_block()

## Mostra a blockchain
print_blockchain()

## 6. Hacker descobre os fatores p e q para gerar o mesmo par de chaves de Alice
(N é um fator público e chaves podem ser reutilizadas)

In [ ]:
# Hacker com os parametros publicos em maos (N)
# Hacker usa o shor para descobrir os fatores p x q = N
p = shor(alice_N)
q = alice_N // p

# Hacker gerar as chaves a partir desses fatores (Sao iguais a de alice)
hacker_pb, hacker_pv = generate_key(p, q)
while hacker_pb != alice_pb:
    hacker_pb, hacker_pv = generate_key(p, q)
  
print("Chave pública: ", hacker_pb)
print("Chave privada: ", hacker_pv)

## 7. Hacker se passa por Alice e gera transações

In [ ]:
# Cria um bloco se passando por Alice e adiciona na blockchain
hacker_block ={"remetente": "alice", 'destinatario': 'ana', 'valor': '150001'}
send_transaction(hacker_block, hacker_pv, alice_N)


### 7.Minerador adiciona a transação de Alice dentro de um bloco

In [ ]:
# Adiciona seu bloco na blockchain
add_transaction_to_block()

## Mostra a blockchain
print_blockchain()

## 8. Alguem e escolhido aleatoriamente para verfiricar a transacao 

In [ ]:
verificador = choice(names)
verify_transaction(verificador, alice_pb, alice_N)

# Aplicação: Ataque as transações Bitcoin

## 1. Instanciando a blockchain

In [6]:
create_blockchain()
print_blockchain()

Sending transaction:  genesis pays 80 to genesis
Sending transaction:  genesis pays 86 to genesis
Sending transaction:  genesis pays 67 to genesis
Block added

 transactions  : 
	 genesis pays genesis 80
	 genesis pays genesis 86
	 genesis pays genesis 67
 hash  :  3086058d888e330312a1e3b6789abfc398ac668db38c4a2fcfbae53a27db5e10




## 2. Alice gera seu par de chaves

In [7]:
alice_p = 5
alice_q = 11
alice_N = alice_p * alice_q
alice_pb, alice_pv = generate_key(alice_p, alice_q)
while alice_pb == alice_pv:
    alice_pb, alice_pv = generate_key(alice_p, alice_q)

print("Chave pública: ", alice_pb)
print("Chave privada: ", alice_pv)
# Alice gera seu bloco
alice_transaction ={"remetente": "alice", 'destinatario': 'bob', 'valor': '150'}
send_transaction(alice_transaction, alice_pb, alice_N)


Chave pública:  17
Chave privada:  33
Sending transaction:  alice pays 150 to bob


## 3. Algumas transações

In [8]:
# Alguns números primos
prime = [3, 7, 11]

# Alguns nomes
names = ["bob", "jeff", "bill", "alice"]

for i in range(randint(1, 3)):
    # Bob gera seu par de chaves
    p = choice(prime)
    q = choice(prime)
    while p == q:
        q = choice(prime)
    N = p * q
    pb, pv = generate_key(p, q)

    # Bob gera seu bloco
    block = {"remetente": choice(names), 'destinatario': choice(names), 'valor': str(randint(1, 150))}
    send_transaction(block, pv, N)

Sending transaction:  bill pays 71 to bill


## 4. Hacker descobre os fatores da chave publica e duplica transaçôes
(antes da transação ser adicionada no bloco)

In [9]:
# Hacker com os parametros publicos em maos (N)
# Hacker usa o shor para descobrir os fatores p x q = N
p = shor(alice_N)
q = alice_N // p

# Hacker gerar as chaves a partir desses fatores (Sao iguais a de alice)
hacker_pb, hacker_pv = generate_key(p, q)
while hacker_pb != alice_pb:
    hacker_pb, hacker_pv = generate_key(p, q)
  

print("Chave pública: ", hacker_pb)
print("Chave privada: ", hacker_pv)

hacker_transaction = {"remetente": "alice", 'destinatario': 'ana', 'valor': '150001'}
send_transaction(hacker_transaction, hacker_pv, alice_N)

Chave pública:  17
Chave privada:  33
Sending transaction:  alice pays 150001 to ana


## 5. Minerador adiciona a transação de Alice dentro de um ledger


In [10]:
add_transaction_to_block()
print_blockchain()

Block added

 transactions  : 
	 genesis pays genesis 80
	 genesis pays genesis 86
	 genesis pays genesis 67
 hash  :  3086058d888e330312a1e3b6789abfc398ac668db38c4a2fcfbae53a27db5e10

 previous_hash  :  3086058d888e330312a1e3b6789abfc398ac668db38c4a2fcfbae53a27db5e10
 transactions  : 
	 alice pays bob 150
	 bill pays bill 71
	 alice pays ana 150001
 hash  :  342c89feacfe5b7628e8dff790c43b497051fef2ce816daaa8de216ba2806551


## 6. Minerador verifica as transações e legitimiza a transação feita pelo Hacker

In [11]:
verify_transaction('minerador', alice_pb, alice_N)

minerador verifiou a transacao
